1

In [5]:
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 드라이버 경로 설정
driver_path = 'C:/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# 웹사이트 열기
url = 'https://search.naver.com/search.naver?where=news&query=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8&sm=tab_opt&sort=1&photo=0&field=0&pd=4&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3A1d&is_sug_officeid=0&office_category=0&service_area=0'  # 대상 웹사이트 URL로 변경
driver.get(url)

# 페이지 배율 줄이기 (줌 아웃)
driver.execute_script("document.body.style.zoom='50%'")
time.sleep(2)  # 페이지 로드 시간을 위해 잠시 대기

# 페이지 스크롤 끝까지 내리기
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 데이터 수집
news_data = []
news_elements = driver.find_elements(By.CSS_SELECTOR, 'li.bx')

for element in news_elements:
    try:
        link_element = element.find_element(By.CSS_SELECTOR, 'a[href*="https://n.news.naver.com/"]')
        link = link_element.get_attribute('href')
        
        # 제목 수집
        title_element = element.find_element(By.CSS_SELECTOR, 'a.news_tit')
        title = title_element.get_attribute('title')
        
        # 시간 수집 - span.info 요소에서 시간 텍스트 찾기
        time_elements = element.find_elements(By.CSS_SELECTOR, 'span.info')
        time_text = ""
        for time_element in time_elements:
            time_text = time_element.text
            if "분 전" in time_text or "시간 전" in time_text or "일 전" in time_text:
                break

        news_data.append({
            'title': title,
            'time': time_text,
            'link': link
        })
    except Exception as e:
        # 예외 발생 시 무시하고 다음 요소로 넘어감
        continue

# 크롬 드라이버 종료
driver.quit()

# 수집한 데이터를 CSV 파일로 저장
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}.csv'

with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'time', 'link'])
    writer.writeheader()
    writer.writerows(news_data)

print(f'Data saved to {file_name}')


Data saved to 20240710_180717.csv


## 링크 수집


### 되는 것


In [27]:
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

# Chrome 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver_path = 'C:/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 링크 목록을 저장한 CSV 파일 경로
links_file = '0303.csv'  # 'links.csv' 파일은 현재 작업 디렉토리에 위치해야 합니다.

# 링크 목록 읽기
with open(links_file, mode='r', encoding='utf-8-sig') as file:
    reader = csv.DictReader(file)
    links = [{'title': row['title'], 'time': row['time'], 'link': row['link']} for row in reader]

# 수집한 데이터를 저장할 리스트
all_news_data = []

for link_info in links:
    driver.get(link_info['link'])
    time.sleep(2)  # 페이지 로드 시간을 위해 잠시 대기

    # 기사 제목 수집
    title_element = driver.find_element(By.CSS_SELECTOR, 'h2#title_area span')
    title = title_element.text

    # 기사 입력 시간 수집
    time_element = driver.find_element(By.CSS_SELECTOR, 'span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')
    article_time = time_element.get_attribute('data-date-time')

    # 댓글 수 수집
    comment_count_element = driver.find_element(By.CSS_SELECTOR, 'a#comment_count')
    comment_count_text = comment_count_element.text
    comment_count = 0 if 'is_zero' in comment_count_element.get_attribute('class') else int(comment_count_text)

    if comment_count > 0:
        # 댓글 페이지로 이동
        comment_page_url = comment_count_element.get_attribute('href')
        driver.get(comment_page_url)
        
        # 페이지 배율 줄이기 (줌 아웃)
        driver.execute_script("document.body.style.zoom='50%'")
        time.sleep(2)  # 페이지 로드 시간을 위해 잠시 대기

        # 클린봇 해제하여 모든 댓글을 볼 수 있도록 설정
        try:
            cleanbot = driver.find_element(By.CSS_SELECTOR, 'a.u_cbox_cleanbot_setbutton')
            cleanbot.click()
            time.sleep(1)
            
            cleanbot_disable = driver.find_element(By.XPATH, "//input[@id='cleanbot_dialog_checkbox_cbox_module']")
            cleanbot_disable.click()
            time.sleep(1)
            
            cleanbot_confirm = driver.find_element(By.CSS_SELECTOR, 'div.u_cbox_layer_cleanbot2_extra a')
            cleanbot_confirm.click()
            time.sleep(1)
        except:
            pass

        # "더보기" 버튼 클릭을 통한 댓글 로딩
        while True:
            try:
                more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.u_cbox_btn_more"))
                )
                driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                time.sleep(1)
                more_button.click()
                time.sleep(2)
            except (NoSuchElementException, TimeoutException):
                break
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(2)

        # 데이터 수집
        while True:
            try:
                comments = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_contents")
                comment_dates = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_date")
                recommend_counts = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_cnt_recomm")
                unrecommend_counts = driver.find_elements(By.CSS_SELECTOR, ".u_cbox_cnt_unrecomm")
                break
            except StaleElementReferenceException:
                time.sleep(0.5)

        for i in range(len(comments)):
            try:
                comment = comments[i].text
                comment_date = comment_dates[i].text
                recommend_count = recommend_counts[i].text
                unrecommend_count = unrecommend_counts[i].text

                all_news_data.append({
                    "title": title,
                    "time": article_time,
                    "comment_count": comment_count,
                    "comment": comment,
                    "comment_date": comment_date,
                    "recommend_count": recommend_count,
                    "unrecommend_count": unrecommend_count
                })
            except StaleElementReferenceException:
                continue

# 크롬 드라이버 종료
driver.quit()

# 수집한 데이터를 CSV 파일로 저장
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}.csv'  # 파일을 현재 작업 디렉토리에 저장

with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = ['title', 'time', 'comment_count', 'comment', 'comment_date', 'recommend_count', 'unrecommend_count']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_news_data)

print(f'Data saved to {file_name}')


Data saved to 20240710_214046.csv
